In [1]:
import json
import pickle
import time
from tqdm import tqdm
import requests

In [2]:
with open("../dataset.pickle", "rb") as fp:
    dataset = pickle.load(fp)
    fp.close()

In [3]:
word_to_embedding = {}

In [4]:
with open("../numberbatch-en.txt", "r", encoding="utf8") as fp:
    line = fp.readline()
    line = fp.readline()
    while line:
        line_tokens = line.split(" ")
        word = line_tokens[0]
        emb = [float(num) for num in line_tokens[1:]]
        word_to_embedding[word.strip()] = emb
        line = fp.readline()
        
    

In [5]:
dataset["id_to_word"] = {wid: word for word, wid in dataset["word_to_id"].items()}
dataset["id_to_word_relation"] = {rid: relation for relation, rid in dataset["word_relation_to_id"].items()}

In [6]:
dataset.keys()

dict_keys(['entity_kg_clean', 'entity_to_id', 'movie_ids', 'relation_counts', 'relation_to_id', 'word_kg_clean', 'word_relation_to_id', 'word_to_id', 'id_to_entity', 'id_to_relation', 'entity_to_edge', 'id_to_word', 'id_to_word_relation'])

In [7]:
len(dataset["word_kg_clean"])

118479

In [8]:
wid_to_new_wids = {}
words = list(dataset["word_to_id"].keys())
for word in tqdm(words):
    wid = dataset["word_to_id"][word]
    word = word.split("/")[3]
    if "_" in word:
        for w in word.split("_"):
            if w not in word_to_embedding:
                continue
            if w in dataset["word_to_id"]:
                new_wid = dataset["word_to_id"][w]
            else:
                new_wid = len(dataset["word_to_id"]) + 1
                dataset["word_to_id"][w] = new_wid
                dataset["id_to_word"][new_wid] = w
            if wid not in wid_to_new_wids:
                wid_to_new_wids[wid] = set()
            wid_to_new_wids[wid].add(new_wid)
    else:
        if word in dataset["word_to_id"]:
            new_wid = dataset["word_to_id"][word]
        else:
            new_wid = len(dataset["word_to_id"]) + 1
            dataset["word_to_id"][word] = new_wid
            dataset["id_to_word"][new_wid] = word
        if wid not in wid_to_new_wids:
            wid_to_new_wids[wid] = set()
        wid_to_new_wids[wid].add(new_wid)
########################        
for wid in wid_to_new_wids:
    word = dataset["id_to_word"].pop(wid)
    dataset["word_to_id"].pop(word)
#########################        
num_edges = len(dataset["word_kg_clean"])
for i in range(num_edges):
    w1id, rid, w2id = dataset["word_kg_clean"][i]
    new_w1s = set([w1id])
    if w1id in wid_to_new_wids:
        new_w1s = wid_to_new_wids[w1id]
    new_w2s = set([w2id])
    if w2id in wid_to_new_wids:
        new_w2s = wid_to_new_wids[w2id]
    dataset["word_kg_clean"][i] = None
    for new_w1 in new_w1s:
        for new_w2 in new_w2s:
            dataset["word_kg_clean"].append((new_w1, rid, new_w2))
dataset["word_kg_clean"] = [edge for edge in dataset["word_kg_clean"] if edge is not None]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91326/91326 [00:01<00:00, 66267.76it/s]


In [9]:
words_to_remove = set()
for word in dataset["word_to_id"]:
    if word.startswith("/c/en") or word not in word_to_embedding:
        words_to_remove.add(dataset["word_to_id"][word])
###############
for wid in words_to_remove:
    word = dataset["id_to_word"].pop(wid)
    dataset["word_to_id"].pop(word)
print(len(words_to_remove))
##############
num_edges = len(dataset["word_kg_clean"])
for i in range(num_edges):
    w1id, rid, w2id = dataset["word_kg_clean"][i]
    if w1id in words_to_remove:
        dataset["word_kg_clean"][i] = None
    if w2id in words_to_remove:
        dataset["word_kg_clean"][i] = None
dataset["word_kg_clean"] = [edge for edge in dataset["word_kg_clean"] if edge is not None]

5773


In [10]:
len(dataset["word_kg_clean"])

149026

In [11]:
dataset["word_to_embedding"] = word_to_embedding

In [12]:
new_wid = 1
word_to_id = {}
id_to_word = {}
relation_to_id = {}
id_to_relation = {}
for i in range(len(dataset["word_kg_clean"])):
    w1id, rid, w2id = dataset["word_kg_clean"][i]
    w1 = dataset["id_to_word"][w1id]
    w2 = dataset["id_to_word"][w2id]
    r = dataset["id_to_word_relation"][rid]
    if w1 in word_to_id:
        new_w1id = word_to_id[w1]
    else:
        new_w1id = len(word_to_id) + 1
        word_to_id[w1] = new_w1id
        id_to_word[new_w1id] = w1
    
    if w2 in word_to_id:
        new_w2id = word_to_id[w2]
    else:
        new_w2id = len(word_to_id) + 1
        word_to_id[w2] = new_w2id
        id_to_word[new_w2id] = w2
        
    if r in relation_to_id:
        new_rid = relation_to_id[r]
    else:
        new_rid = len(relation_to_id) + 1
        relation_to_id[r] = new_rid
        id_to_relation[new_rid] = r  
    dataset["word_kg_clean"][i] = (new_w1id, new_rid, new_w2id)
dataset["word_to_id"] = word_to_id
dataset["id_to_word"] = id_to_word
dataset["word_relation_to_id"] = relation_to_id
dataset["id_to_word_relation"] = id_to_relation

In [13]:
for word in dataset["word_to_id"]:
    if word not in word_to_embedding:
        print(word)

In [14]:
with open("../dataset_word_cleaned.pickle", "wb") as fp:
    pickle.dump(dataset, fp)
    fp.close()